In [1]:
import pandas as pd
import os
import ast

In [10]:
dfc=pd.read_csv("SI tables/SI table 4.tsv",sep="\t",index_col=0)
prots=[x.split(";")[1] for x in dfc.q_name]

In [9]:
#query ncbi for meta data file including expression support for each protein 
os.mkdir("expression_support")
for accession in prots:
    
    !wget -O "expression_support/$accession" "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id=$accession&report=gene_table&format=text"


URL transformed to HTTPS due to an HSTS policy
--2025-07-23 19:05:16--  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id=XP_034487048.1&report=gene_table&format=text
Resolving eutils.ncbi.nlm.nih.gov... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to eutils.ncbi.nlm.nih.gov|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘expression_support/XP_034487048.1’

expression_support/     [ <=>                ]  12.48K  --.-KB/s    in 0.02s   

2025-07-23 19:05:16 (801 KB/s) - ‘expression_support/XP_034487048.1’ saved [12780]

URL transformed to HTTPS due to an HSTS policy
--2025-07-23 19:05:16--  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id=XP_042220148.1&report=gene_table&format=text
Resolving eutils.ncbi.nlm.nih.gov... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to eutils.ncbi.nlm.nih.gov|130.14.29.110|:443... connected.
HTTP request sent, awaiting respons

In [11]:

# Sample data string (replace this with your actual data)
for index,row in dfc.iterrows():
    x=row.q_name.split(";")[1]  
    # Split the data into lines
    lines = open(f"expression_support/{x}","r").readlines()

    for i in range(len(lines)):
        l=lines[i]
        if "pct_ab_initio" in l:
            dfc.loc[index,"pct_ab_initio"]=ast.literal_eval(lines[i+1].strip().split()[-1])
        if "CDS support" in l:
            dfc.loc[index,"CDS_support"]=lines[i+1].strip().split()[-1]

In [12]:
df_rnaseq=dfc.loc[:,["HGT_intervals","Meta_intervals",'q_name','pct_ab_initio','CDS_support']]


df_rnaseq["CDS_support"]=df_rnaseq["CDS_support"].replace('"','')

In [13]:
df_rnaseq['q_name']=[x.split(";")[1] for x in df_rnaseq.q_name]

In [16]:
df_rnaseq.to_csv("SI tables/rnaseq_support.tsv",sep="\t")

In [15]:
df_rnaseq.sort_values

,HGT_intervals,Meta_intervals,q_name,pct_ab_initio,CDS_support
cluster_id,,,,,
1,"[(412,553)]","[(152,234)]",XP_034487048.1,0.0,"""full"""
2,"[(352,559)]","[(187,301)]",XP_042220148.1,0.0,"""full"""
3,"[(127,249)]","[(2,65)]",XP_035715507.1,0.0,"""full"""
4,"[(158,738)]","[(26,70)]",XP_044763649.1,0.0,"""full"""
5,"[(26,316)]","[(356,410)]",XP_054161842.1,0.0,"""full"""
...,...,...,...,...,...
100,"[(350,532)]","[(33,92)]",XP_034245505.1,9.0,NaN
101,"[(28,328)]","[(889,999)]",XP_031633908.1,0.0,"""full"""
102,"[(936,1886)]","[(467,935)]",XP_055306475.1,48.0,NaN
